# Climate Risk and Impact Assessment Dashboard for US Counties

## Documentation

### Python Files Required
<b>climate_risk_webscrape.py</b>: This script automates the process of scraping climate risk data using the selenium library. It sets up a web browser, navigates to the climate risk page, extracts data from an HTML table, and saves the scraped information into a CSV file named "climate_risk_table.csv". This file is then used in this jupyter notebook for analysis and visualization. The CSV file has also been provided for direct use.

### Notebook Information
<b>Climate Dashboard.ipynb</b>: This is the main jupyter notebook containing the analysis and visualization of the dashboard. There are four code blocks to execute. The details about their significance and the functions they contain are given squentially below:
   > <b>Code Block 1</b>: All the required libraries (mentioned in the user guide) are imported to the program. The scrapped CSV file is then pulled and cleaned using various pandas function to ensure there are no irregularities. This includes splitting the county names into state abbreviations and county columns and creating unqiue identifiers for each row ('ID' column) in the <i>df</i> dataframe. This also creates five risk categories using the 'Score' column. We also pull the spatial data from the shapefile "cb_2020_us_county_500k.shp" and convert it to a usable dataframe <i>map</i> using geopandas. Using the state abbreviation and county names, we create a unique identifier ('ID' column) that is consistent with the unique identifiers of <i>df</i>. The two dataframes are then merged into a new dataframe called <i>mergeMap</i>. We also pull the latitude and longitude values from "us-county-boundaries.csv" into the <i>latlong</i> dataframe.\
    \
   > <b>Code Block 2</b>: This code block contains the <i>temppredict</i> function which uses the openmeteo api and pulls various historical climate data for the user-selected county using the latitude and longitude values as parameters from the <i>latlong</i> dataframe. The historical data are then analysed using multiple linear regression to predict the future temperature data for the selected county. The function returns the historical data, predicted data, combined index and trendline obtained from the regression.\
\
    > <b>Code Block 3</b>: This code block contains the <i>aqitrend</i> function which uses the openmeteo api and pulls the live air quality data for the user-selected county using the latitude and longitude values as parameters from the <i>latlong</i> dataframe. The function then returns the daily average data for PM10 and PM2.5 levels (as dataframe) along with their respective dates for tick values in the chart.\
\
    > <b> Code Block 4 </b>: This code block is used to build and execute the dashboard for our visualization. Using the dash library, we build a layout of the dashboard using HTML elements and create spaces for 5 charts to be displayed along with some markdown text. @app.callback is a callable functionality is used to specify the user's input and make interactive output elements. Following this, they are updated using update functions. We first create a dropdown user interface where the user can select the state and the county to observe using the <i>update_counties</i> function. This data is then is used to create another callback for data visualization which utilizes the update_charts function. This function uses the state and county inputs as parameters. Its full purpose is given below:
    >> <u>Markdown</u>: From the user input, it uses markdown text to provide initial details about the county using markdown language using the risk score data. The percentile and risk categories are calculated here before returning the text. It is returned as "markdown_content".\
    >> <u>Bar Chart</u>: The bar chart contains the weighted risk score from for all the counties in the state along with the national and state mean values for comparison. The selected county is highlighted. In the code this is returned as "fig".\
    >> <u>Pie Chart</u>: This shows how the risk is distributed among different climate indicators. It is returned as "figpie".\
    >> <u>Choropleth Map</u>: This map contains the spatial data of the counties in the selected state along with the selected county highlighted with a bold outline. The 'Score' column (displayed in the legend) is used to determine the color of the marker for each county. It is returned as "fig2".\
    >> <u>Temperature Prediction Chart</u>: This calls the <i>temppredict</i> function that yields the historical and predicted data along with the trendline. The data is visualized here after conversion to the Fahrenheit scale. It is returned as "fig3".\
    >> <u>Air Quality Chart</u>: This calls the <i>aqitrend</i> function that yields the live average inhalable pariculate matter data in PM-10 and PM-2.5 levels and extends upto the last three months. The chart is displayed as a subplot containing two line charts placed vertically over the date to show the trend along with the safe levels. It is returned as "fig4".



In [1]:
import pandas as pd
import numpy as np
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import geopandas as gpd
import openmeteo_requests
import requests_cache
from retry_requests import retry
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from plotly.subplots import make_subplots
from scipy.stats import percentileofscore

df = pd.read_csv('climate_risk_table.csv')
df['County_State'] = df['County'].str.split(', ')
county_names = df['County_State'].tolist()
counties = []
states = []
for i in county_names:
    counties.append(i[0])
    states.append(i[1])
df['County'] = counties
df['State'] = states

#Clean the scrapped file
df['County']= df['County'].str.replace(' County','')
df = df.sort_values(by=['State', 'County'])
df['County']= df['County'].str.replace(' Parish','')
df['County']= df['County'].str.replace('Ã±','ñ')

#Attribute the risk categories
conditions = [
    (df['Score'] <= 7),
    (df['Score'] > 7) & (df['Score'] <= 14),
    (df['Score'] > 14) & (df['Score'] <= 21),
    (df['Score'] > 21) & (df['Score'] <= 28),
    (df['Score'] > 28)
]

choices = ['lowest risk', 'low risk', 'medium risk', 'high risk', 'highest risk']
df['Risk'] = np.select(conditions, choices)


#Create unique identifiers to merge with spatial data
df['ID'] = df['State'] + '_' + df['County']
df = df.drop(['County_State'], axis=1)


#Load spatial data and merge with the file
map = gpd.read_file('cb_2020_us_county_500k/cb_2020_us_county_500k.shp')
map.loc[map['NAMELSAD'].str.endswith(' city'), 'NAME'] = map['NAMELSAD']
map['ID']= map['STUSPS'] + '_' + map['NAME']

mergeMap = pd.merge(map, df, on='ID',how='left')
mergeMap = mergeMap.sort_values('ID')
mergeMap = mergeMap.set_index('GEOID')

#Remove areas for which data is not available
no_loc = ['Alaska','Puerto Rico','Hawaii','United States Virgin Islands', 'American Samoa', 'Commonwealth of the Northern Mariana Islands']
mergeMap = mergeMap[~mergeMap['STATE_NAME'].isin(no_loc)]

#Get latitude and longitude details
temp = gpd.read_file('us-county-boundaries.csv')
latlong = temp[['GEOID','INTPTLAT','INTPTLON']].copy()

In [2]:
#Average Temperature Prediction from Open-Meteo API
def temppredict(lat, lng):
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)
    
    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
    	"latitude": lat,
    	"longitude": lng,
    	"start_date": "2013-01-01",
    	"end_date": "2023-12-31",
    	"daily": ["weather_code", "temperature_2m_max", "temperature_2m_min", "temperature_2m_mean", "daylight_duration", "sunshine_duration", "precipitation_sum", "rain_sum", "snowfall_sum", "precipitation_hours", "wind_speed_10m_max", "wind_gusts_10m_max", "wind_direction_10m_dominant", "shortwave_radiation_sum", "et0_fao_evapotranspiration"]
    }
    responses = openmeteo.weather_api(url, params=params)
    
    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    
    # Process daily data. The order of variables needs to be the same as requested.
    daily = response.Daily()
    daily_weather_code = daily.Variables(0).ValuesAsNumpy()
    daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
    daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
    daily_temperature_2m_mean = daily.Variables(3).ValuesAsNumpy()
    daily_daylight_duration = daily.Variables(4).ValuesAsNumpy()
    daily_sunshine_duration = daily.Variables(5).ValuesAsNumpy()
    daily_precipitation_sum = daily.Variables(6).ValuesAsNumpy()
    daily_rain_sum = daily.Variables(7).ValuesAsNumpy()
    daily_snowfall_sum = daily.Variables(8).ValuesAsNumpy()
    daily_precipitation_hours = daily.Variables(9).ValuesAsNumpy()
    daily_wind_speed_10m_max = daily.Variables(10).ValuesAsNumpy()
    daily_wind_gusts_10m_max = daily.Variables(11).ValuesAsNumpy()
    daily_wind_direction_10m_dominant = daily.Variables(12).ValuesAsNumpy()
    daily_shortwave_radiation_sum = daily.Variables(13).ValuesAsNumpy()
    daily_et0_fao_evapotranspiration = daily.Variables(14).ValuesAsNumpy()
    
    daily_data = {"date": pd.date_range(
    	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
    	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
    	freq = pd.Timedelta(seconds = daily.Interval()),
    	inclusive = "left"
    )}
    daily_data["weather_code"] = daily_weather_code
    daily_data["temperature_2m_max"] = daily_temperature_2m_max
    daily_data["temperature_2m_min"] = daily_temperature_2m_min
    daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
    daily_data["daylight_duration"] = daily_daylight_duration
    daily_data["sunshine_duration"] = daily_sunshine_duration
    daily_data["precipitation_sum"] = daily_precipitation_sum
    daily_data["rain_sum"] = daily_rain_sum
    daily_data["snowfall_sum"] = daily_snowfall_sum
    daily_data["precipitation_hours"] = daily_precipitation_hours
    daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
    daily_data["wind_gusts_10m_max"] = daily_wind_gusts_10m_max
    daily_data["wind_direction_10m_dominant"] = daily_wind_direction_10m_dominant
    daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum
    daily_data["et0_fao_evapotranspiration"] = daily_et0_fao_evapotranspiration
    
    daily_dataframe = pd.DataFrame(data = daily_data)
    # Assuming df is your original DataFrame
    daily_dataframe['month'] = daily_dataframe['date'].dt.to_period('M')  # Create a 'month' column based on the date
    
    # Dropping 'date' and 'weather_code' from the calculation
    monthly_avg = daily_dataframe.groupby('month').mean().drop(columns=['weather_code'])
    monthly_avg = monthly_avg.drop(columns=['date'])
    
    # Assume 'temperature_2m_mean' is the target variable and the others are features
    X = monthly_avg.drop(columns=['temperature_2m_mean'])  # Independent variables
    y = monthly_avg['temperature_2m_mean']  # Dependent variable
    
    # Check for missing values and handle them if needed
    monthly_avg.fillna(monthly_avg.mean(), inplace=True)
    
    # Convert any non-numeric columns (if needed)
    X = pd.get_dummies(X, drop_first=True)
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Initialize the linear regression model
    model = LinearRegression()
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = model.predict(X_test)
    
    # Evaluate the model using mean squared error or R-squared
    mse = mean_squared_error(y_test, y_pred)
    r2 = model.score(X_test, y_test)
    
    # Assuming you have future data to predict for the next 60 months, create that DataFrame
    # For simplicity, we simulate this future data with placeholders (you'd need real or averaged data)
    future_data = X.tail(60)  # Just as a placeholder, use actual values for future months
    
    # Predict for the next 60 months
    future_predictions = model.predict(future_data)
    
    # Convert predictions to a DataFrame or Series
    future_predictions_df = pd.DataFrame(future_predictions, columns=['predicted_temperature_2m_mean'])

    # Assuming your DataFrame is called monthly_avg and 'month' is the index
    # Convert the PeriodIndex to DatetimeIndex
    monthly_avg.index = monthly_avg.index.to_timestamp()
    
    # Assuming your original DataFrame is called monthly_avg and 'month' is the index
    # Convert the index to datetime if necessary
    if not isinstance(monthly_avg.index, pd.DatetimeIndex):
        monthly_avg.index = pd.to_datetime(monthly_avg.index, format='%Y-%m')
    
    # Assuming the new predicted DataFrame is called future_predictions_df, and it contains 60 months of predictions
    # Create a date range for the next 60 months based on the last date in monthly_avg
    future_dates = pd.date_range(start=monthly_avg.index[-1] + pd.DateOffset(months=1), periods=60, freq='M')
    future_predictions_df.index = future_dates
    
    # Combine the indices of both historical and future data for the trend line
    combined_index = monthly_avg.index.append(future_predictions_df.index)
    
    # Convert both historical and future dates to ordinal (numerical) format for the trend line
    x_combined = combined_index.map(pd.Timestamp.toordinal).values
    y_combined = np.concatenate((monthly_avg['temperature_2m_mean'].values, future_predictions_df['predicted_temperature_2m_mean'].values))
    
    # Fitting a linear regression to get the extended trend line
    coefficients = np.polyfit(x_combined, y_combined, 1)
    extended_trendline = np.polyval(coefficients, x_combined)
    return monthly_avg, future_predictions_df, combined_index, extended_trendline

In [3]:
#Live AQI Trend
def aqitrend(lat, lng):
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)
    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://air-quality-api.open-meteo.com/v1/air-quality"
    params = {
    	"latitude": lat,
    	"longitude": lng,
    	"hourly": ["pm10", "pm2_5"],
    	"past_days": 92
    }
    responses = openmeteo.weather_api(url, params=params)
    
    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
    print(f"Elevation {response.Elevation()} m asl")
    print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")
    
    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    hourly_pm10 = hourly.Variables(0).ValuesAsNumpy()
    hourly_pm2_5 = hourly.Variables(1).ValuesAsNumpy()
    
    hourly_data = {"date": pd.date_range(
    	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
    	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
    	freq = pd.Timedelta(seconds = hourly.Interval()),
    	inclusive = "left"
    )}
    hourly_data["pm10"] = hourly_pm10
    hourly_data["pm2_5"] = hourly_pm2_5
    
    hourly_dataframe = pd.DataFrame(data = hourly_data)
    print(hourly_dataframe)
        
    # Step 1: Convert the 'date' column to datetime if it isn't already
    hourly_dataframe['date'] = pd.to_datetime(hourly_dataframe['date'])
    
    # Step 2: Create a new column 'date_only' to store only the date (without hours)
    hourly_dataframe['date_only'] = hourly_dataframe['date'].dt.date
    
    # Step 3: Group by 'date_only' and calculate the daily average of 'pm10' and 'pm2_5'
    daily_avg = hourly_dataframe.groupby('date_only').mean().reset_index()
    
    # Step 4: Convert 'date_only' back to datetime for plotting
    daily_avg['date_only'] = pd.to_datetime(daily_avg['date_only'])
    
    # Step 5: Get the min and max values of the 'date_only' column for the x-axis limits
    min_date = daily_avg['date_only'].min()
    max_date = daily_avg['date_only'].max()
    
    # Step 6: Get the maximum values of pm10 and pm2_5 and add 10 to them
    max_pm10 = daily_avg['pm10'].max() + 5
    max_pm2_5 = daily_avg['pm2_5'].max() + 5
    return daily_avg, min_date, max_date, max_pm10, max_pm2_5

In [4]:
#Initialize and run the web-based interface
app = Dash(__name__)

#Define the structure and appearance of the application using HTML and Dash components.
app.layout = html.Div([
    dcc.Dropdown(
        id='state-dropdown',
        options=[{'label': state, 'value': state} for state in mergeMap['STATE_NAME'].unique()],
        value='Pennsylvania'
    ),
    dcc.Dropdown(id='county-dropdown'),
    dcc.Markdown(id='county-info'),
    dcc.Graph(id='score-chart'),
    dcc.Graph(id='factors-pie'),
    dcc.Graph(id='choropleth-map'),
    dcc.Graph(id='timeseries'),
    dcc.Graph(id='aqi'),
])

#Update parts of the application in response to user interactions
@app.callback(
    Output('county-dropdown', 'options'),
    Output('county-dropdown', 'value'),
    Input('state-dropdown', 'value')
)

#Update the counties based on user selection

def update_counties(selected_state):
    filtered_df = mergeMap[mergeMap['STATE_NAME'] == selected_state]
    options = [{'label': county, 'value': county} for county in filtered_df['County']]
    return options, options[0]['value']

@app.callback(
    Output('county-info', 'children'),
    Output('score-chart', 'figure'),
    Output('factors-pie','figure'),
    Output('choropleth-map', 'figure'),
    Output('timeseries', 'figure'),
    Output('aqi', 'figure'),
    Input('state-dropdown', 'value'),
    Input('county-dropdown', 'value'),
)

# Create and return charts for visualization
def update_charts(selected_state, selected_county):
    viz_df = mergeMap[mergeMap['STATE_NAME'] == selected_state] #Filter dataframe
    bar_df = viz_df.drop_duplicates(subset='ID', keep='first')
    bar_df = viz_df.sort_values(by=['STATE_NAME', 'Score', 'County'])
    colors = ['#fee08b' if county != selected_county else '#fc8d59' for county in bar_df['County']]
    nat_mean = mergeMap['Score'].mean()
    state_mean = bar_df['Score'].mean()
    
    # Bar chart
    fig = go.Figure(data=[
        go.Bar(
            x=bar_df['County'],
            y=bar_df['Score'].astype(int),
            marker_color=colors,
            hovertemplate='Risk of %{x}: %{y}<extra></extra>'
        )
    ])
    fig.add_trace(go.Scatter(
        x=bar_df['County'],
        y=[nat_mean] * len(bar_df),
        mode='lines',
        name='National Mean',
        hovertemplate=f'<b>National Mean: {nat_mean:.2f}<extra></extra>',
        line=dict(color='#d53e4f', dash='dash')
    ))
    fig.add_trace(go.Scatter(
        x=bar_df['County'],
        y=[state_mean] * len(bar_df),
        mode='lines',
        name='State Mean',
        hovertemplate=f'<b>State Mean: {state_mean:.2f}<extra></extra>',
        line=dict(color='#3288bd', dash='dash')
    ))
    fig.update_xaxes(
    showticklabels=False,
    showgrid=False,
    zeroline=False)
    
    fig.update_layout(
        title=f'Risk Score of Counties in {selected_state}',
        yaxis_title='Score',
        xaxis_title = 'Counties',
        height=500,
        showlegend = False,
        paper_bgcolor = "rgba(0, 0, 0, 0)",
        plot_bgcolor = "rgba(0, 0, 0, 0)"
    )

    #Risk Factors Pie chart
    county_df = viz_df[viz_df['County'] == selected_county]
    colors = ['#fc8d59','#e6f598','#fee08b','#3288bd','#d53e4f','#99d594']
    piecols = ['Heat', 'Wet Bulb', 'Farm Crop Yields', 'Sea Level Rise','Very Large Fires', 'Economic Damages']
    pievals = county_df[piecols].sum()
    figpie = go.Figure(data=[go.Pie(labels=piecols, values=pievals)])
    figpie.update_traces(hoverinfo='label+value', texttemplate="<b>%{label}</b><br>%{percent}", textfont_size=12, textfont_color = "black",
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)),showlegend=False)
    figpie.update_layout(title=f'Distribution of Risk Factors for {selected_county} County',margin={"r": 250, "t": 50, "l": 50, "b": 50})
    
    #Choropleth Map
    
    fig2 = go.Figure(data=go.Choropleth(
        geojson = viz_df.__geo_interface__,
        locations=viz_df.index,
        z = viz_df['Score'].astype(float),
        zmin = 0,
        zmax = 35,
        text = viz_df['NAME'],
        colorscale = 'YlOrRd',
        colorbar_title = "Risk Score",
        marker_line_color='black',
        marker_line_width=0.5,
        customdata=viz_df[['County', 'Score']]
    ))
    fig2.update_traces(
        hovertemplate="<b>%{customdata[0]}</b><br>Risk Score: %{customdata[1]:.0f}<extra></extra>")
    fig2.add_choropleth(
        geojson = viz_df[viz_df['NAME'] == selected_county].__geo_interface__,
        locations=viz_df[viz_df['NAME'] == selected_county].index,
        z = [viz_df.loc[viz_df['NAME'] == selected_county]['Score'].values[0]],
        zmin = 0,
        zmax = 35,
        text = selected_county,
        colorscale = 'YlOrRd',
        showscale=False,
        marker_line_color='black',
        marker_line_width=2,
        customdata=[viz_df[viz_df['NAME'] == selected_county]['County'].values],
        hovertemplate="<b>%{customdata[0]}</b><br>Risk Score: %{z}<extra></extra>"
    )
    fig2.update_geos(fitbounds="locations", visible=False)
    fig2.update_layout(
        title_text = f'Spatial Risk Map of Counties in {selected_state}', margin={"r": 300, "t": 50, "l": 50, "b": 50})

    viz_df = viz_df.join(latlong.set_index('GEOID'))
    lat = viz_df.loc[viz_df['NAME'] == selected_county]['INTPTLAT'].values[0]
    lng = viz_df.loc[viz_df['NAME'] == selected_county]['INTPTLON'].values[0]
    
    #Temperature Prediction using Machine Learning
    monthly_avg, future_predictions_df, combined_index, extended_trendline = temppredict(lat,lng)

    fig3 = go.Figure()
    
    # Plot the historical data
    fig3.add_trace(go.Scatter(
        x=monthly_avg.index,
        y=(9/5)*monthly_avg['temperature_2m_mean']+32,
        mode='lines+markers',
        name='Mean Temperature (Historical)',
        line=dict(color='#3288bd')
    ))
    
    # Add the predicted temperature_2m_mean values
    fig3.add_trace(go.Scatter(
        x=future_predictions_df.index,
        y=(9/5)*future_predictions_df['predicted_temperature_2m_mean']+32,
        mode='lines+markers',
        name='Predicted Mean Temperature',
        line=dict(color='#99d594')
    ))
    
    # Plot the extended trendline
    fig3.add_trace(go.Scatter(
        x=combined_index,
        y=(9/5)*extended_trendline+32,
        mode='lines',
        name='Extended Trend Line',
        line=dict(color='#d53e4f')
    ))
    

    fig3.update_layout(
        title=f'Temperature Prediction for Next 5 Years for {selected_county} county, {selected_state}',
        xaxis_title='Time (Year)',
        yaxis_title='Temperature Mean (°F)',
        xaxis=dict(tickangle=45),
        legend=dict(
        x=1.05,  
        y=1,
        traceorder='normal',
        orientation='v'
    ),
        margin={"r": 300, "t": 50, "l": 50, "b": 50},
        template='plotly_white',
        paper_bgcolor = "rgba(0, 0, 0, 0)",
        plot_bgcolor = "rgba(0, 0, 0, 0)"
    )    

    #AQI Trendline
    daily_avg,min_date,max_date, max_pm10, max_pm2_5 = aqitrend(lat,lng)
    
    # Create subplots: 2 rows, 1 column
    fig4 = make_subplots(rows=2, cols=1, shared_xaxes=True)
    
    fig4.add_trace(
        go.Scatter(
            x=daily_avg['date_only'],
            y=daily_avg['pm10'],
            mode='lines',
            name='PM10 Levels',
            line=dict(color='#3288bd')
        ),
        row=1, col=1
    )
    
    fig4.add_trace(
        go.Scatter(
            x=[min_date, max_date],
            y=[15, 15],
            mode='lines',
            name='Safe Average Annual Level',
            line=dict(color='#d53e4f', dash='dash', width=2.5)
        ),
        row=1, col=1
    )
    
    fig4.add_trace(
        go.Scatter(
            x=daily_avg['date_only'],
            y=daily_avg['pm2_5'],
            mode='lines',
            name='PM2.5 Levels',
            line=dict(color='#fc8d59')
        ),
        row=2, col=1
    )
    
    fig4.add_trace(
        go.Scatter(
            x=[min_date, max_date],
            y=[5, 5],
            mode='lines',
            name='Safe Average Annual Level',
            line=dict(color='#d53e4f', dash='dash', width=2.5),
            showlegend=False
        ),
        row=2, col=1
    )
    
    fig4.update_layout(
        template='plotly_white',
        title_text='Average Daily Inhalable Particulate Matter Over Last 3 Months',
        height=800,
        showlegend=True,
        paper_bgcolor = "rgba(0, 0, 0, 0)",
        plot_bgcolor = "rgba(0, 0, 0, 0)",
        margin={"r": 300, "t": 75, "l": 50, "b": 50}
    )
    
    fig4.update_yaxes(title_text="PM10 Levels", range=[0, max_pm10], row=1, col=1)
    fig4.update_yaxes(title_text="PM2.5 Levels", range=[0, max_pm2_5], row=2, col=1)
    fig4.update_xaxes(title_text="Time", range=[min_date, max_date], row=2, col=1)

    #Percentile and Risk Calculation
    county_score = viz_df.loc[mergeMap['County'] == selected_county]['Score'].values[0]
    score_list = mergeMap['Score'].tolist()
    score_list = [int(score) if not np.isnan(score) else 0 for score in score_list]
    percentile = percentileofscore(score_list, county_score)
    percentile = round(percentile, 0)
    getgeoid = viz_df.loc[mergeMap['County'] == selected_county].index
    risk_level = mergeMap.loc[getgeoid, 'Risk'].values[0]
    markdown_content= f'''
        ### Climate Risk and Impact Assessment Dashboard for US Counties
        {selected_county} county is in percentile **{int(percentile)}** of all national counties for total climate risk.\n
        {selected_county} county is considered to be *{risk_level}*.\n 
    '''

    return markdown_content, fig, figpie, fig2, fig3, fig4

if __name__ == '__main__':
    app.run_server(debug=True)